In [1]:
%use krangl, lets-plot

import kotlin.random.Random

In [2]:
operator fun List<Double>.plus(v: List<Double>): List<Double> =
    this.zip(v).map { it.first + it.second }

operator fun List<Double>.minus(v: List<Double>): List<Double> =
    this.zip(v).map { it.first - it.second }

operator fun List<Double>.times(x: Double): List<Double> =
    this.map { it * x }

operator fun Double.times(v: List<Double>): List<Double> =
    v * this

fun List<Double>.dot(v: List<Double>): Double =
    this.zip(v).map { it.first * it.second }.reduce { a, b -> a + b }

fun List<List<Double>>.dot(mat: List<List<Double>>): List<List<Double>> {
    val n = this.size
    val m = mat[0].size
    val transpose = List(mat[0].size) { i -> List(mat.size) { j -> mat[j][i] } }
    return List(n) { i -> List(m) { j -> this[i].dot(transpose[j]) } }
}

In [3]:
val LINEAR_KERNEL = { u: List<Double>, v: List<Double> -> u.dot(v) }
val POLY_KERNEL = { beta: Double -> { u: List<Double>, v: List<Double> -> u.dot(v).pow(beta) } }
val GAUSS_KERNEL = { beta: Double -> { u: List<Double>, v: List<Double> -> exp(-beta * (u - v).dot(u - v))} }

In [4]:
class SVM(val epochs: Int = 10, val kernel: (List<Double>, List<Double>) -> Double = LINEAR_KERNEL, val c: Double = 1.0) {
    
    var x: List<List<Double>> = emptyList()
    var y: List<Double> = emptyList()
    var alphas: MutableList<Double> = mutableListOf()
    var bias = 0.0
    
    fun predict(xi: List<Double>): Double {
        val kernels = List(x.size) { i -> kernel(x[i], xi) }
        val pred = alphas.zip(y).map { it.first * it.second }.dot(kernels) + bias
        return pred
    }
    
    fun predictClass(xi: List<Double>): Double {
        return if (predict(xi) >= 0.0) 1.0 else -1.0
    }
    
    fun error(ind: Int): Double {
        val xi = x[ind]
        val yi = y[ind]
        return predict(xi) - yi
    }
    
    fun fit(xToFit: List<List<Double>>, yToFit: List<Double>) {
        x = xToFit
        y = yToFit
        alphas = MutableList(x.size) { 0.0 }
        
        val kernelValues = List(x.size) { i -> List(x.size) { j ->  kernel(x[i], x[j]) } }
        
        for (epoch in 0 until epochs) {
            for (objInd1 in x.indices) {
                var objInd2 = objInd1
                while (objInd2 == objInd1) {
                    objInd2 = Random.nextInt(x.size)
                }
                
                val eta = 2.0 * kernelValues[objInd1][objInd2] - (kernelValues[objInd1][objInd1] + kernelValues[objInd2][objInd2])
                if (eta >= 0.0) {
                    continue
                }
                
                var lower = 0.0
                var upper = 0.0
                if (y[objInd1] == y[objInd2]) {
                   lower = max(0.0, alphas[objInd1] + alphas[objInd2] - c)
                   upper = min(c, alphas[objInd1] + alphas[objInd2])
                } else {
                    lower = max(0.0, alphas[objInd1] - alphas[objInd2])
                    upper = min(c, c + alphas[objInd1] - alphas[objInd2])
                }
                
                val objError2 = error(objInd2)
                val objError1 = error(objInd1)
               
                val oldAlpha1 = alphas[objInd1]
                val oldAlpha2 = alphas[objInd2]
                
                alphas[objInd1] -= (y[objInd1] * (objError2 - objError1)) / eta
                alphas[objInd1] = min(alphas[objInd1], upper)
                alphas[objInd1] = max(alphas[objInd1], lower)
                alphas[objInd2] += y[objInd2] * y[objInd1] * (oldAlpha1 - alphas[objInd1])
                
                val bias2 = bias - objError2 - y[objInd2] * (alphas[objInd2] - oldAlpha2) * kernelValues[objInd2][objInd2]
                    - y[objInd1] * (alphas[objInd1] - oldAlpha1) * kernelValues[objInd1][objInd2]
                val bias1 = bias - objError1 - y[objInd1] * (alphas[objInd1] - oldAlpha1) * kernelValues[objInd1][objInd1]
                    - y[objInd2] * (alphas[objInd2] - oldAlpha2) * kernelValues[objInd1][objInd2]
                
                bias = when {
                    0.0 < alphas[objInd2] && alphas[objInd2] < c -> bias2
                    0.0 < alphas[objInd1] && alphas[objInd1] < c -> bias1
                    else -> (bias2 + bias1) / 2.0
                }
            }
        }
    }
}

var df = DataFrame.readCSV("chips.csv")
var x = List(df.nrow) { i -> List(df.ncol - 1) { j -> df[j][i].toString().toDouble() } }
var y = List(df.nrow) { i -> if (df[df.ncol - 1][i].toString() == "P") 1.0 else -1.0 }

val C = listOf(0.05, 0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0)
val D = listOf(2.0, 3.0, 4.0, 5.0)
val BETA = listOf(1.0, 2.0, 3.0, 4.0, 5.0)


// --------------- LINEAR KERNEL --------------------

var maxAccuracy = 0.0
var bestSvm = SVM()
var bestC = 0.0

for (c in C) {
    val svm = SVM(1, LINEAR_KERNEL, c)

    val confusionMatrix = List(2) { MutableList(2) { 0.0 } }
    for (i in x.indices) {
        val xWithoutObj = x.toMutableList()
        val yWithoutObj = y.toMutableList()
        xWithoutObj.removeAt(i)
        yWithoutObj.removeAt(i)

        svm.fit(xWithoutObj, yWithoutObj)
        val obj = x[i]
        val objClass = y[i]
        val predClass = svm.predictClass(obj)
        val cmI = if (predClass == 1.0) 0 else 1
        val cmJ = if (objClass == 1.0) 0 else 1
        confusionMatrix[cmI][cmJ]++
    }
    println(confusionMatrix)
    val accuracy = (confusionMatrix[0][0] + confusionMatrix[1][1]) / confusionMatrix.sumByDouble { it.sum() }
    if (maxAccuracy < accuracy) {
        maxAccuracy = accuracy
        bestSvm = svm
        bestC = c
    }
}

println("Linear kernel:\n    Accuracy: $maxAccuracy\n    C: $bestC\n")
bestSvm.fit(x, y)

val cellCount = 50
var xs = List(cellCount * cellCount) { i -> -1.1 + 2.2 * ((i % cellCount).toDouble() / (cellCount - 1)) }
var ys = List(cellCount * cellCount) { i -> -1.1 + 2.2 * ((i / cellCount).toDouble() / (cellCount - 1)) }

var z = xs.zip(ys).map { 
    bestSvm.predictClass(listOf(it.first, it.second))
}

var plotInfo = mapOf(
    "x" to x.map { it.first() },
    "y" to x.map { it[1] },
    "color" to y
)

var p1 = lets_plot {x=xs; y=ys; fill=z}
p1 += geom_tile() + 
    scale_fill_hue() + 
    geom_point {  x=plotInfo["x"]; y=plotInfo["y"]; fill=x; color=plotInfo["color"] }
p1.show()

// --------------- POLY KERNEL --------------------
maxAccuracy = 0.0
bestSvm = SVM()
bestC = 0.0

var bestD = 0.0

for (d in D) {
    for (c in C) {
        val svm = SVM(1, POLY_KERNEL(d), c)

        val confusionMatrix = List(2) { MutableList(2) { 0.0 } }
        for (i in x.indices) {
            val xWithoutObj = x.toMutableList()
            val yWithoutObj = y.toMutableList()
            xWithoutObj.removeAt(i)
            yWithoutObj.removeAt(i)

            svm.fit(xWithoutObj, yWithoutObj)
            val obj = x[i]
            val objClass = y[i]
            val predClass = svm.predictClass(obj)
            val cmI = if (predClass == 1.0) 0 else 1
            val cmJ = if (objClass == 1.0) 0 else 1
            confusionMatrix[cmI][cmJ]++
        }
        println(confusionMatrix)
        val accuracy = (confusionMatrix[0][0] + confusionMatrix[1][1]) / confusionMatrix.sumByDouble { it.sum() }
        if (maxAccuracy < accuracy) {
            maxAccuracy = accuracy
            bestSvm = svm
            bestC = c
            bestD = d
        }
    }
}

println("Poly kernel:\n    Accuracy: $maxAccuracy\n    C: $bestC\n    D: $bestD")

bestSvm.fit(x, y)

z = xs.zip(ys).map { 
    bestSvm.predictClass(listOf(it.first, it.second))
}

var p2 = lets_plot {x=xs; y=ys; fill=z}
p2 += geom_tile() + 
    scale_fill_hue() + 
    geom_point {  x=plotInfo["x"]; y=plotInfo["y"]; fill=x; color=plotInfo["color"] }
p2.show()


// --------------- GAUSS_KERNEL --------------------
maxAccuracy = 0.0
bestSvm = SVM()
bestC = 0.0

var bestBeta = 0.0

for (beta in BETA) {
    for (c in C) {
        val svm = SVM(1, GAUSS_KERNEL(beta), c)

        val confusionMatrix = List(2) { MutableList(2) { 0.0 } }
        for (i in x.indices) {
            val xWithoutObj = x.toMutableList()
            val yWithoutObj = y.toMutableList()
            xWithoutObj.removeAt(i)
            yWithoutObj.removeAt(i)

            svm.fit(xWithoutObj, yWithoutObj)
            val obj = x[i]
            val objClass = y[i]
            val predClass = svm.predictClass(obj)
            val cmI = if (predClass == 1.0) 0 else 1
            val cmJ = if (objClass == 1.0) 0 else 1
            confusionMatrix[cmI][cmJ]++
        }
        println(confusionMatrix)
        val accuracy = (confusionMatrix[0][0] + confusionMatrix[1][1]) / confusionMatrix.sumByDouble { it.sum() }
        if (maxAccuracy < accuracy) {
            maxAccuracy = accuracy
            bestSvm = svm
            bestC = c
            bestBeta = beta
        }
    }
}

println("Gauss kernel:\n    Accuracy: $maxAccuracy\n    C: $bestC\n    Beta: $bestBeta")
bestSvm.fit(x, y)

z = xs.zip(ys).map { 
    bestSvm.predictClass(listOf(it.first, it.second))
}

var p3 = lets_plot {x=xs; y=ys; fill=z}
p3 += geom_tile() + 
    scale_fill_hue() + 
    geom_point {  x=plotInfo["x"]; y=plotInfo["y"]; fill=x; color=plotInfo["color"] }
p3.show()

    
df = DataFrame.readCSV("geyser.csv")
x = List(df.nrow) { i -> List(df.ncol - 1) { j -> df[j][i].toString().toDouble() } }
y = List(df.nrow) { i -> if (df[df.ncol - 1][i].toString() == "P") 1.0 else -1.0 }


// --------------- LINEAR KERNEL --------------------

maxAccuracy = 0.0
bestSvm = SVM()
bestC = 0.0

for (c in C) {
    val svm = SVM(1, LINEAR_KERNEL, c)

    val confusionMatrix = List(2) { MutableList(2) { 0.0 } }
    for (i in x.indices) {
        val xWithoutObj = x.toMutableList()
        val yWithoutObj = y.toMutableList()
        xWithoutObj.removeAt(i)
        yWithoutObj.removeAt(i)

        svm.fit(xWithoutObj, yWithoutObj)
        val obj = x[i]
        val objClass = y[i]
        val predClass = svm.predictClass(obj)
        val cmI = if (predClass == 1.0) 0 else 1
        val cmJ = if (objClass == 1.0) 0 else 1
        confusionMatrix[cmI][cmJ]++
    }
    println(confusionMatrix)
    val accuracy = (confusionMatrix[0][0] + confusionMatrix[1][1]) / confusionMatrix.sumByDouble { it.sum() }
    if (maxAccuracy < accuracy) {
        maxAccuracy = accuracy
        bestSvm = svm
        bestC = c
    }
}

println("Linear kernel:\n    Accuracy: $maxAccuracy\n    C: $bestC\n")
bestSvm.fit(x, y)


xs = List(cellCount * cellCount) { i -> 25 * ((i % cellCount).toDouble() / (cellCount - 1)) }
ys = List(cellCount * cellCount) { i -> 1.0 + 5.0 * ((i / cellCount).toDouble() / (cellCount - 1)) }

z = xs.zip(ys).map { 
    bestSvm.predictClass(listOf(it.first, it.second))
}

plotInfo = mapOf(
    "x" to x.map { it.first() },
    "y" to x.map { it[1] },
    "color" to y
)

var p4 = lets_plot {x=xs; y=ys; fill=z}
p4 += geom_tile() + 
    scale_fill_hue() + 
    geom_point {  x=plotInfo["x"]; y=plotInfo["y"]; fill=x; color=plotInfo["color"] }
p4.show()

// --------------- POLY KERNEL --------------------
maxAccuracy = 0.0
bestSvm = SVM()
bestC = 0.0

bestD = 0.0

for (d in D) {
    for (c in C) {
        val svm = SVM(1, POLY_KERNEL(d), c)

        val confusionMatrix = List(2) { MutableList(2) { 0.0 } }
        for (i in x.indices) {
            val xWithoutObj = x.toMutableList()
            val yWithoutObj = y.toMutableList()
            xWithoutObj.removeAt(i)
            yWithoutObj.removeAt(i)

            svm.fit(xWithoutObj, yWithoutObj)
            val obj = x[i]
            val objClass = y[i]
            val predClass = svm.predictClass(obj)
            val cmI = if (predClass == 1.0) 0 else 1
            val cmJ = if (objClass == 1.0) 0 else 1
            confusionMatrix[cmI][cmJ]++
        }
        println(confusionMatrix)
        val accuracy = (confusionMatrix[0][0] + confusionMatrix[1][1]) / confusionMatrix.sumByDouble { it.sum() }
        if (maxAccuracy < accuracy) {
            maxAccuracy = accuracy
            bestSvm = svm
            bestC = c
            bestD = d
        }
    }
}

println("Poly kernel:\n    Accuracy: $maxAccuracy\n    C: $bestC\n    D: $bestD")

bestSvm.fit(x, y)

z = xs.zip(ys).map { 
    bestSvm.predictClass(listOf(it.first, it.second))
}

var p5 = lets_plot {x=xs; y=ys; fill=z}
p5 += geom_tile() + 
    scale_fill_hue() + 
    geom_point {  x=plotInfo["x"]; y=plotInfo["y"]; fill=x; color=plotInfo["color"] }
p5.show()

// --------------- GAUSS_KERNEL --------------------
maxAccuracy = 0.0
bestSvm = SVM()
bestC = 0.0

bestBeta = 0.0

for (beta in BETA) {
    for (c in C) {
        val svm = SVM(1, GAUSS_KERNEL(beta), c)

        val confusionMatrix = List(2) { MutableList(2) { 0.0 } }
        for (i in x.indices) {
            val xWithoutObj = x.toMutableList()
            val yWithoutObj = y.toMutableList()
            xWithoutObj.removeAt(i)
            yWithoutObj.removeAt(i)

            svm.fit(xWithoutObj, yWithoutObj)
            val obj = x[i]
            val objClass = y[i]
            val predClass = svm.predictClass(obj)
            val cmI = if (predClass == 1.0) 0 else 1
            val cmJ = if (objClass == 1.0) 0 else 1
            confusionMatrix[cmI][cmJ]++
        }
        println(confusionMatrix)
        val accuracy = (confusionMatrix[0][0] + confusionMatrix[1][1]) / confusionMatrix.sumByDouble { it.sum() }
        if (maxAccuracy < accuracy) {
            maxAccuracy = accuracy
            bestSvm = svm
            bestC = c
            bestBeta = beta
        }
    }
}

println("Gauss kernel:\n    Accuracy: $maxAccuracy\n    C: $bestC\n    Beta: $bestBeta")
bestSvm.fit(x, y)

z = xs.zip(ys).map { 
    bestSvm.predictClass(listOf(it.first, it.second))
}

var p6 = lets_plot {x=xs; y=ys; fill=z}
p6 += geom_tile() + 
    scale_fill_hue() + 
    geom_point {  x=plotInfo["x"]; y=plotInfo["y"]; fill=x; color=plotInfo["color"] }
p6.show()

[[16.0, 18.0], [42.0, 42.0]]
[[11.0, 23.0], [47.0, 37.0]]
[[12.0, 20.0], [46.0, 40.0]]
[[12.0, 14.0], [46.0, 46.0]]
[[22.0, 13.0], [36.0, 47.0]]
[[20.0, 18.0], [38.0, 42.0]]
[[18.0, 26.0], [40.0, 34.0]]
[[15.0, 22.0], [43.0, 38.0]]
Linear kernel:
    Accuracy: 0.5847457627118644
    C: 5.0



[[17.0, 9.0], [41.0, 51.0]]
[[13.0, 7.0], [45.0, 53.0]]
[[19.0, 9.0], [39.0, 51.0]]
[[25.0, 12.0], [33.0, 48.0]]
[[36.0, 12.0], [22.0, 48.0]]
[[34.0, 15.0], [24.0, 45.0]]
[[28.0, 15.0], [30.0, 45.0]]
[[32.0, 17.0], [26.0, 43.0]]
[[16.0, 12.0], [42.0, 48.0]]
[[11.0, 7.0], [47.0, 53.0]]
[[18.0, 23.0], [40.0, 37.0]]
[[12.0, 17.0], [46.0, 43.0]]
[[8.0, 21.0], [50.0, 39.0]]
[[11.0, 22.0], [47.0, 38.0]]
[[22.0, 17.0], [36.0, 43.0]]
[[18.0, 22.0], [40.0, 38.0]]
[[11.0, 4.0], [47.0, 56.0]]
[[14.0, 10.0], [44.0, 50.0]]
[[20.0, 7.0], [38.0, 53.0]]
[[25.0, 8.0], [33.0, 52.0]]
[[28.0, 14.0], [30.0, 46.0]]
[[28.0, 14.0], [30.0, 46.0]]
[[28.0, 24.0], [30.0, 36.0]]
[[30.0, 17.0], [28.0, 43.0]]
[[11.0, 10.0], [47.0, 50.0]]
[[11.0, 15.0], [47.0, 45.0]]
[[16.0, 17.0], [42.0, 43.0]]
[[17.0, 18.0], [41.0, 42.0]]
[[19.0, 19.0], [39.0, 41.0]]
[[14.0, 22.0], [44.0, 38.0]]
[[15.0, 21.0], [43.0, 39.0]]
[[13.0, 19.0], [45.0, 41.0]]
Poly kernel:
    Accuracy: 0.711864406779661
    C: 5.0
    D: 2.0


[[9.0, 7.0], [49.0, 53.0]]
[[9.0, 6.0], [49.0, 54.0]]
[[22.0, 5.0], [36.0, 55.0]]
[[21.0, 4.0], [37.0, 56.0]]
[[20.0, 6.0], [38.0, 54.0]]
[[17.0, 7.0], [41.0, 53.0]]
[[20.0, 3.0], [38.0, 57.0]]
[[22.0, 5.0], [36.0, 55.0]]
[[15.0, 7.0], [43.0, 53.0]]
[[18.0, 4.0], [40.0, 56.0]]
[[16.0, 6.0], [42.0, 54.0]]
[[22.0, 8.0], [36.0, 52.0]]
[[18.0, 4.0], [40.0, 56.0]]
[[22.0, 4.0], [36.0, 56.0]]
[[20.0, 3.0], [38.0, 57.0]]
[[18.0, 5.0], [40.0, 55.0]]
[[14.0, 5.0], [44.0, 55.0]]
[[16.0, 6.0], [42.0, 54.0]]
[[19.0, 5.0], [39.0, 55.0]]
[[17.0, 4.0], [41.0, 56.0]]
[[25.0, 6.0], [33.0, 54.0]]
[[23.0, 7.0], [35.0, 53.0]]
[[22.0, 6.0], [36.0, 54.0]]
[[17.0, 5.0], [41.0, 55.0]]
[[21.0, 5.0], [37.0, 55.0]]
[[13.0, 1.0], [45.0, 59.0]]
[[27.0, 6.0], [31.0, 54.0]]
[[21.0, 5.0], [37.0, 55.0]]
[[17.0, 6.0], [41.0, 54.0]]
[[28.0, 8.0], [30.0, 52.0]]
[[27.0, 8.0], [31.0, 52.0]]
[[20.0, 2.0], [38.0, 58.0]]
[[14.0, 5.0], [44.0, 55.0]]
[[17.0, 4.0], [41.0, 56.0]]
[[20.0, 7.0], [38.0, 53.0]]
[[21.0, 3.0], [37.0, 5

[[66.0, 16.0], [22.0, 118.0]]
[[62.0, 11.0], [26.0, 123.0]]
[[63.0, 17.0], [25.0, 117.0]]
[[61.0, 14.0], [27.0, 120.0]]
[[62.0, 19.0], [26.0, 115.0]]
[[67.0, 22.0], [21.0, 112.0]]
[[62.0, 22.0], [26.0, 112.0]]
[[60.0, 17.0], [28.0, 117.0]]
Linear kernel:
    Accuracy: 0.8333333333333334
    C: 0.1



[[53.0, 19.0], [35.0, 115.0]]
[[56.0, 18.0], [32.0, 116.0]]
[[53.0, 20.0], [35.0, 114.0]]
[[49.0, 21.0], [39.0, 113.0]]
[[53.0, 18.0], [35.0, 116.0]]
[[55.0, 19.0], [33.0, 115.0]]
[[51.0, 18.0], [37.0, 116.0]]
[[47.0, 18.0], [41.0, 116.0]]
[[51.0, 21.0], [37.0, 113.0]]
[[48.0, 18.0], [40.0, 116.0]]
[[52.0, 18.0], [36.0, 116.0]]
[[49.0, 25.0], [39.0, 109.0]]
[[48.0, 23.0], [40.0, 111.0]]
[[49.0, 21.0], [39.0, 113.0]]
[[47.0, 24.0], [41.0, 110.0]]
[[46.0, 21.0], [42.0, 113.0]]
[[46.0, 23.0], [42.0, 111.0]]
[[53.0, 27.0], [35.0, 107.0]]
[[49.0, 27.0], [39.0, 107.0]]
[[39.0, 26.0], [49.0, 108.0]]
[[50.0, 19.0], [38.0, 115.0]]
[[45.0, 26.0], [43.0, 108.0]]
[[43.0, 25.0], [45.0, 109.0]]
[[52.0, 23.0], [36.0, 111.0]]
[[51.0, 24.0], [37.0, 110.0]]
[[50.0, 29.0], [38.0, 105.0]]
[[47.0, 30.0], [41.0, 104.0]]
[[54.0, 24.0], [34.0, 110.0]]
[[54.0, 24.0], [34.0, 110.0]]
[[53.0, 32.0], [35.0, 102.0]]
[[52.0, 37.0], [36.0, 97.0]]
[[54.0, 30.0], [34.0, 104.0]]
Poly kernel:
    Accuracy: 0.774774774774

[[66.0, 70.0], [22.0, 64.0]]
[[66.0, 62.0], [22.0, 72.0]]
[[70.0, 6.0], [18.0, 128.0]]
[[67.0, 12.0], [21.0, 122.0]]
[[61.0, 14.0], [27.0, 120.0]]
[[67.0, 23.0], [21.0, 111.0]]
[[70.0, 23.0], [18.0, 111.0]]
[[68.0, 21.0], [20.0, 113.0]]
[[71.0, 59.0], [17.0, 75.0]]
[[74.0, 61.0], [14.0, 73.0]]
[[65.0, 7.0], [23.0, 127.0]]
[[64.0, 7.0], [24.0, 127.0]]
[[65.0, 21.0], [23.0, 113.0]]
[[63.0, 22.0], [25.0, 112.0]]
[[64.0, 25.0], [24.0, 109.0]]
[[66.0, 26.0], [22.0, 108.0]]
[[58.0, 62.0], [30.0, 72.0]]
[[69.0, 66.0], [19.0, 68.0]]
[[63.0, 13.0], [25.0, 121.0]]
[[64.0, 17.0], [24.0, 117.0]]
[[63.0, 19.0], [25.0, 115.0]]
[[62.0, 29.0], [26.0, 105.0]]
[[61.0, 22.0], [27.0, 112.0]]
[[58.0, 19.0], [30.0, 115.0]]
[[69.0, 65.0], [19.0, 69.0]]
[[66.0, 71.0], [22.0, 63.0]]
[[66.0, 15.0], [22.0, 119.0]]
[[62.0, 12.0], [26.0, 122.0]]
[[63.0, 25.0], [25.0, 109.0]]
[[60.0, 20.0], [28.0, 114.0]]
[[61.0, 21.0], [27.0, 113.0]]
[[63.0, 20.0], [25.0, 114.0]]
[[56.0, 73.0], [32.0, 61.0]]
[[64.0, 71.0], [24.0, 